In [ ]:
// This import style lets us update notebooks without having to push a new cargo package
:dep onq = { version = "0.3.0", path = "../" }
:dep plotters = { version = "^0.3.7", default_features = false, features = ["evcxr", "line_series"] }

In [ ]:
use onq::{
    core::{OnqError, QduId, StableState},
    operations::Operation,
    vm::{program::LockType, Instruction, Program, ProgramBuilder, OnqVm}, // Use full VM for potential future extension
    // Note: Simulator could also be used if no classical feedback needed *during* the run
};
use std::f64::consts::PI;

# Quantum Teleportation using Projection-Based Entanglement

This notebook demonstrates Quantum Teleportation using the `onq` library and ONQ-VM.

**Standard Teleportation Review:**
Typically, teleportation requires Alice and Bob to share a pre-established entangled Bell pair (e.g., $|\Phi^+\rangle = \frac{1}{\sqrt{2}}(|00\rangle + |11\rangle)$). Alice interacts the message qubit with her half of the pair, measures her two qubits in the Bell basis, and sends two classical bits to Bob. Bob uses these bits to apply corrective operations (X, Z) to his half of the pair, recovering the original message state. The initial Bell pair is usually created using unitary gates (Hadamard + CNOT).

**ONQ Approach using Projection:**
In this example, we explore using `onq`'s unique `RelationalLock` operation with `establish=true`. Based on our framework, this performs a **non-unitary projection** of Alice's and Bob's initial state ($|00\rangle$) directly onto the desired Bell state ($|\Phi^+\rangle$). This simulates the *achievement* of an integrated, coherent state via projection rather than unitary evolution. The rest of the protocol (Alice's operations, classical communication via registers, Bob's corrections) proceeds similarly using the ONQ-VM.

We will teleport the state $|+\rangle = \frac{1}{\sqrt{2}}(|0\rangle+|1\rangle)$.

In [ ]:
// Helper for QduId creation
fn qid(id: u64) -> QduId { QduId(id) }

// --- Setup ---
let msg_q = qid(0);    // Message QDU (will be prepared in |+>)
let alice_q = qid(1);  // Alice's QDU
let bob_q = qid(2);    // Bob's QDU

println!("QDUs defined: msg={}, alice={}, bob={}", msg_q, alice_q, bob_q);

In [ ]:
// --- Build the Teleportation Program using Projection ---
let mut builder = ProgramBuilder::new();
println!("Building Projection-Based Teleportation Program...");

// 1. Prepare Message state |+>
builder = builder.pb_add(Instruction::QuantumOp(Operation::InteractionPattern {
    target: msg_q,
    pattern_id: "Superposition".to_string(), // H analog -> |+> state
}));
println!("  Step 1: Prepared Message QDU in |+> state.");

// 2. Create Bell Pair |Φ+> between Alice and Bob via Projection
// This is the key difference: using RelationalLock instead of H+CNOT
builder = builder.pb_add(Instruction::QuantumOp(Operation::RelationalLock {
    qdu1: alice_q,
    qdu2: bob_q,
    lock_type: LockType::BellPhiPlus, // Target |Φ+> state
    establish: true,
}));
println!("  Step 2: Created Bell Pair via Projection (RelationalLock).");

// 3. Alice's Bell measurement basis change operations
builder = builder.pb_add(Instruction::QuantumOp(Operation::ControlledInteraction { // CNOT(Msg, Alice)
    control: msg_q,
    target: alice_q,
    pattern_id: "QualityFlip".to_string(),
}));
builder = builder.pb_add(Instruction::QuantumOp(Operation::InteractionPattern { // H on Msg
    target: msg_q,
    pattern_id: "Superposition".to_string(),
}));
println!("  Step 3: Applied Bell Measurement basis change gates.");

// 4. Stabilize Alice's qubits and Record classical results
builder = builder.pb_add(Instruction::Stabilize { targets: vec![msg_q, alice_q] });
builder = builder.pb_add(Instruction::Record { qdu: msg_q,   register: "m_msg".to_string() });   // Classical bit 1
builder = builder.pb_add(Instruction::Record { qdu: alice_q, register: "m_alice".to_string() }); // Classical bit 2
println!("  Step 4: Stabilized Alice's QDUs & added Record instructions.");

// 5. Bob's Classical Corrections (Conditional Operations on bob_q)
// 5a. X Correction based on Alice's measurement (m_alice)
builder = builder.pb_add(Instruction::Label("X_Correction".to_string()));
builder = builder.pb_add(Instruction::BranchIfZero { // If m_alice == 0, jump past X gate
    register: "m_alice".to_string(),
    label: "skip_x_corr".to_string(),
});
builder = builder.pb_add(Instruction::QuantumOp(Operation::InteractionPattern { // Apply X if m_alice == 1
    target: bob_q,
    pattern_id: "QualityFlip".to_string(),
}));
builder = builder.pb_add(Instruction::Label("skip_x_corr".to_string()));

// 5b. Z Correction based on Message measurement (m_msg)
builder = builder.pb_add(Instruction::Label("Z_Correction".to_string()));
builder = builder.pb_add(Instruction::BranchIfZero { // If m_msg == 0, jump past Z gate
    register: "m_msg".to_string(),
    label: "skip_z_corr".to_string(),
});
builder = builder.pb_add(Instruction::QuantumOp(Operation::InteractionPattern { // Apply Z if m_msg == 1
    target: bob_q,
    pattern_id: "PhaseIntroduce".to_string(),
}));
builder = builder.pb_add(Instruction::Label("skip_z_corr".to_string()));
println!("  Step 5: Added Bob's conditional correction logic.");

// 6. Stabilize Bob's qubit to observe the final teleported state
builder = builder.pb_add(Instruction::Stabilize { targets: vec![bob_q] });
builder = builder.pb_add(Instruction::Record { qdu: bob_q, register: "m_bob".to_string() });
println!("  Step 6: Added final stabilization for Bob's QDU.");

// 7. Halt
builder = builder.pb_add(Instruction::Halt);

// Build final program
let program = builder.build().expect("Failed to build program");
println!("Program built successfully.");

In [ ]:
// Print the constructed program for review
println!("\nProjection-Based Teleportation Program Instructions:\n{}", program);

In [ ]:
// --- Run the ONQ-VM ---
let mut vm = OnqVm::new();
println!("Running ONQ-VM...");

match vm.run(&program) {
    Ok(()) => {
        println!("Simulation finished successfully.");
        // --- Analyze Results ---
        println!("\n--- ONQ-VM Execution Finished ---");
        let final_mem = vm.get_classical_memory();
        println!("Final Classical Memory: {:?}", final_mem);

        let m_msg = vm.get_classical_register("m_msg");
        let m_alice = vm.get_classical_register("m_alice");
        let m_bob = vm.get_classical_register("m_bob");

        println!("\nAnalysis:");
        println!("- Alice's measurement outcomes (classical bits sent): msg={}, alice={}", m_msg, m_alice);
        println!("- Bob's final stabilized state: {}", m_bob);
        println!("- Verification Notes:");
        println!("  - Input state for Message QDU was |+>.");
        println!("  - Teleportation *should* transfer |+> state to Bob via projection & corrections.");
        println!("  - Stabilizing the resulting |+> state on Bob's QDU yields a deterministic outcome ({}) based on framework rules.", m_bob);
        println!("  - Comparison: The outcome {} should match the outcome from Example 1 where |+> was stabilized.", m_bob);
        // We previously saw |+> stabilizing to 0 in Example 1, let's assert that for consistency here.
         assert_eq!(m_bob, 0, "Expected Bob's QDU to stabilize to 0, consistent with stabilizing |+>");
         println!("- Success! Bob's outcome matches expectation for teleported |+> state under current stabilization rules.");

    }
    Err(e) => {
        eprintln!("\n--- Simulation Failed ---");
        eprintln!("Error: {}", e);
    }
}